In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

### Scraping de alunos e fóruns - Trilha Data Science

Serão captados dados sobre:
- tags dos posts no fórum
- links para os perfis dos alunos
- nomes dos alunos
- status de resolvido dos posts

In [3]:
alunos = []
links = []
solved = []
tag1 = []
tag2 = []
tag3 = []

for page in range(1500):
    url = 'https://cursos.alura.com.br/forum/categoria-data-science/todos/'+str(page+1)
    
    headers = {}

    response = requests.get(url, headers=headers)
    forum = BeautifulSoup(response.content,'html.parser')
    lista_alunos = forum.find_all('li',{'class':'forumList-item'})

    tags = []

    for aluno in lista_alunos:
        alunos.append(aluno.find('p',{'class': 'forumList-item-info-name'}).find('strong').get_text().strip())
        solved.append(aluno.find('div',{'class': 'forumList-item-icon'}).find('img')['class'][0][15:])
        try:
            links.append('https://cursos.alura.com.br' + aluno.find('a',{'class': 'forumList-item-info'}).get("href"))
        except:
            links.append(' ')
        tags = aluno.find_all('li',{'class': 'topic-breadCrumb-item'})
        tag1.append(tags[0].get_text().strip())
        try:
            tag2.append(tags[1].get_text().strip())
        except:
            tag2.append(' ')
        try:
            tag3.append(tags[2].get_text().strip())   
        except:
            tag3.append(' ')

In [9]:
tabela_alunos_alura = pd.DataFrame([alunos,links,tag1,tag2,tag3,solved]).T
tabela_alunos_alura.columns=['nome_aluno','url_perfil','tag1','tag2','tag3','solved']
tabela_alunos_alura.head()

,nome_aluno,url_perfil,tag1,tag2,tag3,solved
0,Welinton,https://cursos.alura.com.br/user/wgs-gilton,Data Science,Excel,"Análise de dados: cálculos, padrões e estratég...",notsolved
1,Kleber,https://cursos.alura.com.br/user/rodrigueskleb...,Data Science,Machine Learning,Machine Learning: classificação com SKLearn,notsolved
2,Mario,https://cursos.alura.com.br/user/mario-sandrini1,Data Science,Business Intelligence,Power BI Desktop: tratamento de dados no Power...,notsolved
3,Caio,https://cursos.alura.com.br/user/cmayrinkguedes,Data Science,Excel,Funções com Excel: operações matemáticas e fil...,notsolved
4,Joshua,https://cursos.alura.com.br/user/joshua-jka-18,Data Science,NoSQL,MongoDB: conhecendo um banco de dados NoSQL,notsolved


In [10]:
# salvando os dados em csv
tabela_alunos_alura.to_csv('tabela_alunos_datascience.csv')

### Scraping das atividades dos alunos - Trilha Data Science

Serão captados dados sobre:
- nome dos cursos cursados
- datas de início e fim de cada curso
- url do curso
- linkedin do aluno
- url do aluno
- atividades nos últimos 30 dias

In [12]:
# Função que lê uma página de aluno e retorna suas atividades nos últimos 30 dias

def retorna_atividade(url,perfil_aluno):
    
    atividade = perfil_aluno.find('div',{'class': 'pointsGrid'})

    dias_historico = []
    dia_com_atividade = []

    for dia in atividade.find_all('div',{'class': 'pointsGrid-cell'}):
        texto = dia.get_text().strip()
        dias_historico.append(texto[-10:])
        if "ganhou pontos" in texto:
            dia_com_atividade.append(0)
        else:
            dia_com_atividade.append(1)

    atividade_na_plataforma = {'ativo_ultimos_7d': 0,
                              'ativo_ultimos_14d': 0,
                              'ativo_ultimos_21d': 0,
                              'ativo_ultimos_30d': 0,
                              'qte_dias_com_atividades': 0}

    if sum(dia_com_atividade[-7:]) > 0:
        atividade_na_plataforma['ativo_ultimos_7d'] = 1
    if sum(dia_com_atividade[-14:]) > 0:
        atividade_na_plataforma['ativo_ultimos_14d'] = 1
    if sum(dia_com_atividade[-21:]) > 0:
        atividade_na_plataforma['ativo_ultimos_21d'] = 1
    if sum(dia_com_atividade[-30:]) > 0:
        atividade_na_plataforma['ativo_ultimos_30d'] = 1
        
    atividade_na_plataforma['qte_dias_com_atividades'] = sum(dia_com_atividade)

    aux = pd.DataFrame([atividade_na_plataforma])
    aux['url_aluno'] = url

    return aux

In [22]:
urls_alunos = list(tabela_alunos_alura['url_perfil'].value_counts().index)[1:]

In [23]:
df_atividades_alunos = pd.DataFrame()

for url in urls_alunos:
    
    nome_curso_aluno = []
    data_inicio_curso_aluno = []
    data_fim_curso_aluno = []
    url_curso_aluno = []
    linkedin_aluno = []
    
    headers = {}
    
    aux = pd.DataFrame()

    try:
        response = requests.get(url, headers=headers)
    except:
        continue
    
    perfil_aluno = BeautifulSoup(response.content, 'html.parser')

    lista_cursos_finalizados = perfil_aluno.find('ul',{'class': 'profile-finishedCourses-list'})
    card = perfil_aluno.find('div',{'class': 'profile-header-avatarAndEdit-wrapper'})

    
    try:
        cursos_finalizados = lista_cursos_finalizados.find_all('li',{'class': 'card-list__item'})

        for curso in cursos_finalizados:
            data_inicio_curso_aluno.append(curso.find('span',{'class': 'course-card__start-date'}).get_text().strip()[16:])
            data_fim_curso_aluno.append(curso.find('span',{'class': 'course-card__finish-date'}).get_text().strip()[21:])
            nome_curso_aluno.append(curso.find('span',{'class': 'course-card__name'}).get_text().strip())
            url_curso_aluno.append(curso.find('a',{'class': 'course-card__course-link'}).get("href"))
            try:
                linkedin_aluno.append(card.find('a',{'class': 'profile-link profile-LINKEDIN'}).get("href"))
            except:
                linkedin_aluno.append(' ')
                
    
        aux = pd.DataFrame([nome_curso_aluno,data_inicio_curso_aluno,data_fim_curso_aluno,url_curso_aluno,linkedin_aluno]).T
        aux['url_aluno'] = url
        aux = pd.merge(aux,retorna_atividade(url,perfil_aluno),how='left',on='url_aluno')

        df_atividades_alunos = pd.concat((df_atividades_alunos,aux),axis = 0)
    
    except:
        continue

In [27]:
# renomeando colunas
df_atividades_alunos.columns = ['nome_Curso','data_inicio_curso','data_fim_curso','url_curso','linkedin','url_aluno',
                                'ativo_ultimos_7d','ativo_ultimos_14d','ativo_ultimos_21d','ativo_ultimos_30d','qte_dias_com_atividades']

In [28]:
df_atividades_alunos.head()

,nome_Curso,data_inicio_curso,data_fim_curso,url_curso,linkedin,url_aluno,ativo_ultimos_7d,ativo_ultimos_14d,ativo_ultimos_21d,ativo_ultimos_30d,qte_dias_com_atividades
0,A Empresa Ágil: implemente o Business Agility ...,26/10/2022,26/10/2022,/course/empresa-agil,,https://cursos.alura.com.br/user/pietro-sanchini,0,1,1,1,3
1,Agilidade: promovendo a transformação ágil,26/10/2022,26/10/2022,/course/fundamentos-agilidade,,https://cursos.alura.com.br/user/pietro-sanchini,0,1,1,1,3
2,Business Intelligence: trabalhando com Data Wa...,10/07/2022,12/07/2022,/course/business-intelligence-introducao-intel...,,https://cursos.alura.com.br/user/pietro-sanchini,0,1,1,1,3
3,Clustering: extraindo padrões de dados,19/09/2021,29/09/2021,/course/cluster-analysis,,https://cursos.alura.com.br/user/pietro-sanchini,0,1,1,1,3
4,Data Science: análise de series temporais,12/10/2020,12/10/2020,/course/data-science-series-temporais,,https://cursos.alura.com.br/user/pietro-sanchini,0,1,1,1,3


In [29]:
# salvando em csv
df_atividades_alunos.to_csv('tabela_alunos_datascience_atividades.csv')

### Scraping de outras trilhas

In [31]:
def scrap_alunos(url_forum,qte_paginas):

    alunos = []
    links = []
    tag1 = []
    tag2 = []
    tag3 = []

    for pagina in range(qte_paginas):
        #pagina = 1

        url = url_forum + str(pagina+1)

        headers = {}

        response = requests.get(url, headers=headers)
        forum = BeautifulSoup(response.content, 'html.parser')

        lista_alunos = forum.find_all('li',{'class': 'forumList-item'})

        print(pagina)


        tags = []

        for aluno in lista_alunos:
            alunos.append(aluno.find('p',{'class': 'forumList-item-info-name'}).find('strong').get_text().strip())
            try:
                links.append('https://cursos.alura.com.br' + aluno.find('a',{'class': 'forumList-item-info'}).get("href"))
            except:
                links.append(' ')
            tags = aluno.find_all('li',{'class': 'topic-breadCrumb-item'})
            tag1.append(tags[0].get_text().strip())
            try:
                tag2.append(tags[1].get_text().strip())
            except:
                tag2.append(' ')
            try:
                tag3.append(tags[2].get_text().strip())   
            except:
                tag3.append(' ')


    return pd.DataFrame([alunos,links,tag1,tag2,tag3]).T

In [37]:
urls_foruns = ['https://cursos.alura.com.br/forum/categoria-inovacao-gestao/todos/',
              'https://cursos.alura.com.br/forum/categoria-design-ux/todos/',
              'https://cursos.alura.com.br/forum/categoria-devops/todos/',
              'https://cursos.alura.com.br/forum/categoria-front-end/todos/',
              'https://cursos.alura.com.br/forum/categoria-programacao/todos/',
              'https://cursos.alura.com.br/forum/categoria-mobile/todos/']

# quantidade de página do fórum a serem raspadas
qte_paginas = [1251,683,863,2315,5460,598]

# trilhas

# tabela_alunos_gestao = pd.DataFrame()
# tabela_alunos_design = pd.DataFrame()
# tabela_alunos_devops = pd.DataFrame()
# tabela_alunos_front = pd.DataFrame()
# tabela_alunos_programacao = pd.DataFrame()

# for url, qte in zip(urls_foruns,qte_paginas):
#     tabela_alunos_gestao = scrap_alunos(url,qte)
#     tabela_alunos_design = scrap_alunos(url,qte)
#     tabela_alunos_devops = scrap_alunos(url,qte)
#     tabela_alunos_front = scrap_alunos(url,qte)
#     tabela_alunos_programacao = scrap_alunos(url,qte)


In [38]:
# tabela_alunos_gestao.to_csv('tabela_alunos_gestao.csv')
# tabela_alunos_design.to_csv('tabela_alunos_design.csv')
# tabela_alunos_devops.to_csv('tabela_alunos_devops.csv')
# tabela_alunos_front.to_csv('tabela_alunos_front.csv')
# tabela_alunos_programacao.to_csv('tabela_alunos_programacao.csv')

### Scraping de atividades dos alunos das outras trilhas

In [40]:
def get_atividades_por_aluno(urls_alunos):

    df_atividades_alunos = pd.DataFrame()

    for url in urls_alunos:

        nome_curso_aluno = []
        data_inicio_curso_aluno = []
        data_fim_curso_aluno = []
        url_curso_aluno = []
        linkedin_aluno = []

        headers = {}

        aux = pd.DataFrame()

        try:
            response = requests.get(url, headers=headers)
        except:
            continue

        perfil_aluno = BeautifulSoup(response.content, 'html.parser')

        lista_cursos_finalizados = perfil_aluno.find('ul',{'class': 'profile-finishedCourses-list'})
        card = perfil_aluno.find('div',{'class': 'profile-header-avatarAndEdit-wrapper'})

        #print(url)

        try:
            cursos_finalizados = lista_cursos_finalizados.find_all('li',{'class': 'card-list__item'})

            for curso in cursos_finalizados:
                data_inicio_curso_aluno.append(curso.find('span',{'class': 'course-card__start-date'}).get_text().strip()[16:])
                data_fim_curso_aluno.append(curso.find('span',{'class': 'course-card__finish-date'}).get_text().strip()[21:])
                nome_curso_aluno.append(curso.find('span',{'class': 'course-card__name'}).get_text().strip())
                url_curso_aluno.append(curso.find('a',{'class': 'course-card__course-link'}).get("href"))
                try:
                    linkedin_aluno.append(card.find('a',{'class': 'profile-link profile-LINKEDIN'}).get("href"))
                except:
                    linkedin_aluno.append(' ')


            aux = pd.DataFrame([nome_curso_aluno,data_inicio_curso_aluno,data_fim_curso_aluno,url_curso_aluno,linkedin_aluno]).T
            aux['url_aluno'] = url
            aux = pd.merge(aux,retorna_atividade(url,perfil_aluno),how='left',on='url_aluno')

            df_atividades_alunos = pd.concat((df_atividades_alunos,aux),axis = 0)
            

        except:
            continue

    df_atividades_alunos.columns = ['nome_Curso','data_inicio_curso','data_fim_curso','url_curso','linkedin','url_aluno',
                                    'ativo_ultimos_7d','ativo_ultimos_14d','ativo_ultimos_21d','ativo_ultimos_30d',
                                           'qte_dias_com_atividades']
    return df_atividades_alunos

In [42]:
# atividades_alunos_gestao = get_atividades_por_aluno(list(tabela_alunos_gestao[1].value_counts().index)[1:])
# atividades_alunos_design = get_atividades_por_aluno(list(tabela_alunos_design[1].value_counts().index)[1:])
# atividades_alunos_devops = get_atividades_por_aluno(list(tabela_alunos_devops[1].value_counts().index)[1:])
# atividades_alunos_front = get_atividades_por_aluno(list(tabela_alunos_front[1].value_counts().index)[1:])
# atividades_alunos_programacao = get_atividades_por_aluno(list(tabela_alunos_programacao[1].value_counts().index)[1:])

In [43]:
# atividades_alunos_gestao.to_csv('tabela_alunos_gestao_atividades.csv')
# atividades_alunos_design.to_csv('tabela_alunos_design_atividades.csv')
# atividades_alunos_devops.to_csv('tabela_alunos_devops_atividades.csv')
# atividades_alunos_front.to_csv('tabela_alunos_front_atividades.csv')
# atividades_alunos_programacao.to_csv('tabela_alunos_programacao_atividades.csv')

### Scraping de cursos

In [120]:
url = 'https://www.alura.com.br/cursos-online-design-ux'

headers = {}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

In [121]:
tipos_cursos = soup.find_all('div',{'class': 'subcategoria'})

In [122]:
assuntos = []

for tipo in tipos_cursos:
    assuntos.append(tipo.find('h2',{'class': 'subcategoria__titulo'}).get_text())

In [123]:
nome_cursos = []

for tipo in tipos_cursos:
    lista = tipo.find('ul',{'class': 'subcategoria__lista'})
    for curso in lista.find_all('li',{'class': 'subcategoria__item'}):
        a = tipo.find('h2',{'class': 'subcategoria__titulo'}).get_text()
        b = curso.find('span',{'class': 'card-curso__nome'}).get_text()
        link = 'https://www.alura.com.br' + curso.find('a',{'class': 'card-curso'}).get("href")
        carga = curso.find('span',{'class': 'card-curso__carga'}).get_text().strip()
        nome_cursos.append([a,b,carga,link])

In [124]:
info_curso =  []

for i in range(len(nome_cursos)):
    url = nome_cursos[i][3]
    
    headers = {}

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    card = soup.find('div',{'class': 'course-container--icon'})

    
    aux = []

    for i in card.find_all('p',{'class': 'courseInfo-card-wrapper-infos'}):
        aux.append(i.get_text().strip())

    matriculados = int(aux[1])
    try:
        nota = float(aux[2])
    except:
        continue
    info_curso.append([url,matriculados,nota])
    
    

In [125]:
df_info_cursos = pd.DataFrame(info_curso)
df_info_cursos.columns = ['url','contagem_alunos','nota_curso']

df_nome_cursos = pd.DataFrame(nome_cursos)
df_nome_cursos.columns = ['Assunto','Nome_Curso','Carga_Horaria', 'url']

df_cursos_alura = pd.merge(df_nome_cursos,df_info_cursos,how='inner',on = 'url')
df_cursos_alura 

,Assunto,Nome_Curso,Carga_Horaria,url,contagem_alunos,nota_curso
0,UI Design,Curso Adobe XD: design visual de um site mobile,10h,https://www.alura.com.br/curso-online-adobe-xd...,9549,9.3
1,UI Design,Curso Adobe XD: refinando o visual da interface,10h,https://www.alura.com.br/curso-online-adobe-xd...,2279,9.3
2,UI Design,Curso Adobe XD: trabalhando com microinterações,10h,https://www.alura.com.br/curso-online-xd-micro...,1431,9.4
3,UI Design,Curso Adobe XD: componentes da interface,10h,https://www.alura.com.br/curso-online-adobe-xd...,3251,9.3
4,UI Design,Curso Sketch: incrementando e melhorando a UI,08h,https://www.alura.com.br/curso-online-sketch-i...,252,9.2
...,...,...,...,...,...,...
283,Fotografia,Curso LightRoom: usando seu celular para edita...,08h,https://www.alura.com.br/curso-online-lightroo...,902,9.3
284,Fotografia,Curso Photoshop: edite suas fotos pelo celular,06h,https://www.alura.com.br/curso-online-photosho...,524,9.4
285,Áudio (Produção e Pós-Produção),Curso Produção de áudio,10h,https://www.alura.com.br/curso-online-producao...,1456,9.2
286,Áudio (Produção e Pós-Produção),Curso Produção de Podcasts: da concepção à pub...,08h,https://www.alura.com.br/curso-online-producao...,3729,9.2


In [126]:
df_cursos_alura.to_excel('base_cursos_alura_design.xlsx')